# 包含智谱的embedding 和 其他开源的embedding

In [1]:
# 智谱测试成功
import os
from zhipuai import ZhipuAI
def zhipu_embedding(text: str):

    api_key = os.environ['ZHIPUAI_API_KEY']
    client = ZhipuAI(api_key=api_key)
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response

text = '要生成 embedding 的输入文本，字符串形式。'
response = zhipu_embedding(text=text)

In [2]:
print(f'response类型为：{type(response)}')
print(f'embedding类型为：{response.object}')
print(f'生成embedding的model为：{response.model}')
print(f'生成的embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为: {response.data[0].embedding[:10]}')

response类型为：<class 'zhipuai.types.embeddings.EmbeddingsResponded'>
embedding类型为：list
生成embedding的model为：embedding-2
生成的embedding长度为：1024
embedding（前10）为: [0.01789241097867489, 0.0644201934337616, -0.009342828765511513, 0.027074744924902916, 0.004067827481776476, -0.055978596210479736, -0.042238060384988785, -0.030031980946660042, -0.016357745975255966, 0.06777039915323257]


In [3]:
# huggingface 上的开源embedding模型
# moka-ai/m3e-base
# 读出markdown文件
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("/workspaces/Medical_Chat/knowledge_db/test.md")
md_pages = loader.load()
print(f"载入后的变量类型为：{type(md_pages)}，",  f"该 Markdown 一共包含 {len(md_pages)} 页")


[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


载入后的变量类型为：<class 'list'>， 该 Markdown 一共包含 1 页


In [4]:
md_page = md_pages[0]
print(f"每一个元素的类型：{type(md_page)}.", 
    f"该文档的描述性数据：{md_page.metadata}", 
    f"查看该文档的内容:\n{md_page.page_content[0:][:200]}", 
    sep="\n------\n")


每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '/workspaces/Medical_Chat/knowledge_db/test.md'}
------
查看该文档的内容:
恭喜您完成了本书第一单元内容的学习！

总的来说，在第一部分中，我们学习并掌握了关于 Prompt 的两个核心原则：

编写清晰具体的指令；
如果适当的话，给模型一些思考时间。
您还学习了迭代式 Prompt 开发的方法，并了解了如何找到适合您应用程序的 Prompt 的过程是非常关键的。

我们还讨论了大型语言模型的许多功能，包括摘要、推断、转换和扩展。您也学习了如何搭建个性化的聊天机器人。在第


In [7]:
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
for page in md_pages:
    page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), page.page_content)
    print(page.page_content)


恭喜您完成了本书第一单元内容的学习！总的来说，在第一部分中，我们学习并掌握了关于 Prompt 的两个核心原则：编写清晰具体的指令；
如果适当的话，给模型一些思考时间。
您还学习了迭代式 Prompt 开发的方法，并了解了如何找到适合您应用程序的 Prompt 的过程是非常关键的。我们还讨论了大型语言模型的许多功能，包括摘要、推断、转换和扩展。您也学习了如何搭建个性化的聊天机器人。在第一部分中，您的收获应该颇丰，希望通过第一部分学习能为您带来愉悦的体验。我们期待您能灵感迸发，尝试创建自己的应用。请大胆尝试，并分享给我们您的想法。您可以从一个微型项目开始，或许它具备一定的实用性，或者仅仅是一项有趣的创新。请利用您在第一个项目中得到的经验，去创造更优秀的下一项目，以此类推。如果您已经有一个宏大的项目设想，那么，请毫不犹豫地去实现它。最后，希望您在完成第一部分的过程中感到满足，感谢您的参与。我们热切期待着您的惊艳作品。接下来，我们将进入第二部分的学习！


In [34]:
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader, Document
import re


class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata


loader = UnstructuredMarkdownLoader("/workspaces/Medical_Chat/knowledge_db/test.md")
md_pages = loader.load()

print(f"载入后的变量类型为：{type(md_pages)}，",  f"该 Markdown 一共包含 {len(md_pages)} 页")
md_page = md_pages[0]
print(f"每一个元素的类型：{type(md_page)}.", 
    f"该文档的描述性数据：{md_page.metadata}", 
    f"查看该文档的内容:\n{md_page.page_content[0:][:200]}", 
    sep="\n------\n")

pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
for page in md_pages:
    page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), page.page_content)
    print(page.page_content)

CHUNK_SIZE = 20
OVERLAP_SIZE = 5

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)

split_docs = [Document(page_content=doc, metadata=md_page.metadata) for doc in text_splitter.split_text(md_page.page_content)]
embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")
persist_directory = 'WHYembedding/vector_db'

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory
)

载入后的变量类型为：<class 'list'>， 该 Markdown 一共包含 1 页
每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '/workspaces/Medical_Chat/knowledge_db/test.md'}
------
查看该文档的内容:
恭喜您完成了本书第一单元内容的学习！

总的来说，在第一部分中，我们学习并掌握了关于 Prompt 的两个核心原则：

编写清晰具体的指令；
如果适当的话，给模型一些思考时间。
您还学习了迭代式 Prompt 开发的方法，并了解了如何找到适合您应用程序的 Prompt 的过程是非常关键的。

我们还讨论了大型语言模型的许多功能，包括摘要、推断、转换和扩展。您也学习了如何搭建个性化的聊天机器人。在第
恭喜您完成了本书第一单元内容的学习！总的来说，在第一部分中，我们学习并掌握了关于 Prompt 的两个核心原则：编写清晰具体的指令；
如果适当的话，给模型一些思考时间。
您还学习了迭代式 Prompt 开发的方法，并了解了如何找到适合您应用程序的 Prompt 的过程是非常关键的。我们还讨论了大型语言模型的许多功能，包括摘要、推断、转换和扩展。您也学习了如何搭建个性化的聊天机器人。在第一部分中，您的收获应该颇丰，希望通过第一部分学习能为您带来愉悦的体验。我们期待您能灵感迸发，尝试创建自己的应用。请大胆尝试，并分享给我们您的想法。您可以从一个微型项目开始，或许它具备一定的实用性，或者仅仅是一项有趣的创新。请利用您在第一个项目中得到的经验，去创造更优秀的下一项目，以此类推。如果您已经有一个宏大的项目设想，那么，请毫不犹豫地去实现它。最后，希望您在完成第一部分的过程中感到满足，感谢您的参与。我们热切期待着您的惊艳作品。接下来，我们将进入第二部分的学习！


In [35]:
# print(f"向量库中存储的数量：{vectordb._collection.count()}")
question = "聊天"
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")
# print(sim_docs[0])
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")


检索到的内容数：3
检索到的第0个内容: 
搭建个性化的聊天机器
--------------
检索到的第1个内容: 
学习了如何搭建个性化的聊天机器人。在第一
--------------
检索到的第2个内容: 
的聊天机器人。在第一
--------------


In [ ]:
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma
import re

loader = UnstructuredMarkdownLoader("/workspaces/Medical_Chat/knowledge_db/test.md")
md_pages = loader.load()

print(f"载入后的变量类型为：{type(md_pages)}，",  f"该 Markdown 一共包含 {len(md_pages)} 页")
md_page = md_pages[0]
print(f"每一个元素的类型：{type(md_page)}.", 
    f"该文档的描述性数据：{md_page.metadata}", 
    f"查看该文档的内容:\n{md_page.page_content[0:][:200]}", 
    sep="\n------\n")

pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
for page in md_pages:
    page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), page.page_content)
    print(page.page_content)

CHUNK_SIZE = 500
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)

split_docs = [UnstructuredMarkdownLoader(page_content=doc) for doc in text_splitter.split_text(md_page.page_content)]

embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")
persist_directory = 'WHYembedding/vector_db'

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory
)

# 不同科室的数据放在不同的数据库

In [1]:
import os
import sys
import re
from pathlib import Path
import tempfile
from embedding.call_embedding import get_embedding
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
# 首先我只处理pdf文件


In [12]:
import errno
import os
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


DEFAULT_DB_PATH = "/workspaces/Medical_Chat/WHYmedicalbooks"
DEFAULT_PERSIST_PATH = "/workspaces/Medical_Chat/WHYembedding"

def get_files(dir_path):
    file_list = []
    for filepath, dirnames, filenames in os.walk(dir_path):
        for filename in filenames:
            if filename.endswith('.pdf'):  # 确保只处理PDF文件
                file_list.append(os.path.join(filepath, filename))
    return file_list

def create_db(files):
    if files is None:
        return "can't load empty file"
    if not isinstance(files, list):
        files = [files]

    for file_path in files:
        print(f"正在处理文件：{file_path}")
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()  # 假设这会返回一个包含文档内容的列表
        
        # 切分文档
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
        split_docs = text_splitter.split_documents(docs)
        
        # 获取嵌入向量
        embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")
        
        # 为每个文件的父目录创建唯一的持久化路径
        parent_dir_name = Path(file_path).parent.stem

        persist_dir = os.path.join(DEFAULT_PERSIST_PATH, parent_dir_name, "vector_db")
        
        print(f"持久化路径为：{persist_dir}")
        if not os.path.exists(persist_dir):
            print
            os.makedirs(persist_dir, exist_ok=True)  # 创建持久化目录
        

                # Check if the directory exists and has write permissions
        if not os.access(persist_dir, os.W_OK):
            try:
                os.makedirs(persist_dir, exist_ok=True)  # Try to create the directory
            except OSError as e:
                if e.errno != errno.EEXIST:  # Ignore error if directory already exists
                    raise
                print(f"Error: The directory {persist_dir} cannot be written to.")
                continue
            
        # 创建向量数据库
        vectordb = Chroma.from_documents(
            documents=split_docs[:10],
            embedding=embedding,
            persist_directory=persist_dir
        )
        


if __name__ == "__main__":
    # 获取所有子目录中的PDF文件
    pdf_files = get_files(DEFAULT_DB_PATH)
    print(f"共找到{len(pdf_files)}个PDF文件")
    create_db(files=pdf_files)

共找到2个PDF文件
正在处理文件：/workspaces/Medical_Chat/WHYmedicalbooks/临床科/临床流行病学.pdf
持久化路径为：/workspaces/Medical_Chat/WHYembedding/临床科/vector_db


PermissionError: [Errno 13] Permission denied: '/workspaces/Medical_Chat/WHYembedding/临床科'

: 

In [3]:
# 加载向量知识库
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma
embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")
persist_directory = '/workspaces/Medical_Chat/WHYembedding/精神科/vector_db'
# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
print(type(vectordb))
print(f"向量库中存储的数量：{vectordb._collection.count()}")
question = "临床?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n***************************************************\n")


<class 'langchain_community.vectorstores.chroma.Chroma'>
向量库中存储的数量：10
检索到的内容数：3
检索到的第0个内容: 
 目录21一、流行病学233二、病因233三、临床表现234四、病程与预后 236五、诊断与鉴别诊断236六、预防与治疗237第三节孤独症谱系障碍239一、概述239二、流行病学239三、临床表现239四、诊断与鉴别诊断241五、病程及预后 242六、干预和治疗242第四节注意缺陷多动性障碍243一、流行病学244二、病因和病理机制244三、临床表现244四、诊断与鉴别诊断245五、病程和预后246六、治疗和干预247第五节抽动障碍 249一、流行病学 249二、病因和病理机制249三、临床表现249四、诊断与鉴别诊断 250五、病程与预后251六、治疗 251第六节特定性学习障碍252第十三章精神分裂症 256第一节概述 256第二节病因与发病机制257一、遗传因素257二、神经病理学及大脑结构的异常 259三、神经生化方面的异常261四、神经发育不良假说261五、子宫内感染与产伤 262六、社会心理因素263第三节临床表现263第四节诊断与鉴别诊断 264一、诊断标准264二、鉴别诊断 266三、实验室检查 266
***************************************************
检索到的第1个内容: 
 24■目录四、病程和预后 331五、病因和发病机制研究331六、治疗331七、预防和康复 331第四节社交焦虑障碍331一、流行病学 332二、临床表现 332三、诊断与鉴别诊断 332四、病程和预后333五、病因和发病机制研究 333六、治疗 333七、预防和康复334第五节特定恐惧症334一、流行病学 335二、临床表现 335三、诊断与鉴别诊断 335四、病程和预后335五、病因和发病机制研究 336六、治疗 336七、预防和康复336第六节广泛性焦虑障碍337一、流行病学 337二、临床表现 337三、诊断与鉴别诊断337四、病程和预后338五、病因和发病机制研究339六、治疗 339七、预防和康复340第七节分离焦虑障碍340一、流行病学 340二、临床表现341三、诊断与鉴别诊断 341四、病程和预后 341五、病因和发病机制研究

: 